Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [28]:
import requests

server = "http://192.168.1.67/healthconnect/cdr/fhir/r4/"
api_url = server + "metadata"
response = requests.get(api_url)
#response.json()

Find a patient. Simple search for a patient named wrexham

In [29]:
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os
fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")

api_url = server + "Patient?name=wrexham"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': 'd54db26a-873a-40a7-9c5e-0713e9cbc069',
 'type': 'searchset',
 'timestamp': '2025-12-23T06:11:46Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient/18',
   'resource': {'resourceType': 'Patient',
    'address': [{'id': 'qEFQG',
      'line': ['30 PASTURE ROAD', 'BARTON-UPON-HUMBER', 'S HUMBERSIDE'],
      'period': {'start': '2025-06-24'},
      'postalCode': 'DN18 5HU',
      'use': 'home'}],
    'birthDate': '1987-02-18',
    'gender': 'unknown',
    'generalPractitioner': [{'id': 'ZGilw',
      'identifier': {'period': {'start': '2025-03-02'},
       'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
       'value': 'A20047'},
      'reference': 'Organization/1',
      'type': 'Organization'}],
    'id': '18',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/nhs-number',
      'type':

Now lets find diagnostic reports for this patient.

In [30]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': 'e813c66f-005c-414a-a4a9-432341688e71',
 'type': 'searchset',
 'timestamp': '2025-12-23T06:11:46Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?patient=18'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport/22',
   'resource': {'resourceType': 'DiagnosticReport',
    'basedOn': [{'identifier': {'assigner': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
         'value': 'RBS'}},
       'system': 'https://nw-gmsa.github.io/Id/PLAC',
       'type': {'coding': [{'code': 'PLAC',
          'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]},
       'value': '1000152867'},
      'type': 'ServiceRequest'}],
    'category': [{'coding': [{'code': 'GE',
        'system': 'http://terminology.hl7.org/CodeSystem/v2-0074'}]}],
    'code': {'coding': [{'code': 'R125_Cardiology',
       'display': 'PANEL: R125.1, R125.2 -

Can also return details about the organisation that created this.

In [31]:
api_url = server + "Organization/5"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Organization',
 'active': True,
 'id': '5',
 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': '699X0'}],
 'name': 'NORTH WEST GLH LED BY MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
 'partOf': {'display': 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
  'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'R0A'},
  'reference': 'Organization/4'},
 'type': [{'coding': [{'code': 'RO173',
     'display': 'PATHOLOGY LAB',
     'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}],
 'meta': {'lastUpdated': '2025-12-19T11:20:46Z', 'versionId': '2'}}

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [32]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd
import json

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
responseJSON = response.json()

print(responseJSON['total'])

reports = []
reportJ = []
for entry in responseJSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reportJ.append(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

6
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER-Family-follow-up-Rare
R375.1
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-18T12:14:40+00:00
PCR-FLT3-ITD
M80.18
M80.21


In [33]:
print(len(reports))
df1 = pd.DataFrame([vars(s) for s in reports])
df1

6


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,extension,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,7,None,None,<fhirclient.models.meta.Meta object at 0x1166a...,None,None,None
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,16,None,None,<fhirclient.models.meta.Meta object at 0x11669...,None,None,None
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,22,None,None,<fhirclient.models.meta.Meta object at 0x11669...,None,None,None
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,61,None,None,<fhirclient.models.meta.Meta object at 0x11669...,None,None,None
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,75,None,None,<fhirclient.models.meta.Meta object at 0x11643...,None,None,None
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,<fhirclient.models.fhirreference.FHIRReference...,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,88,None,None,<fhirclient.models.meta.Meta object at 0x11650...,None,None,None


In [34]:
newJson = json.dumps(reportJ)
df2 = pd.read_json(newJson)
df2

/var/folders/16/bplm70c55mj7020_2tms62sh0000gn/T/ipykernel_71875/1748824183.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2 = pd.read_json(newJson)


,resourceType,basedOn,category,code,id,identifier,issued,performer,presentedForm,resultsInterpreter,status,subject,meta,encounter
0,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...","{'coding': [{'code': 'R125_Cardiology', 'displ...",7,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-10-14T15:59:16+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Jonathan Edgerley'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-19T12:22:14Z', 'versi...",NaN
1,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...","{'coding': [{'code': 'R125_Cardiology', 'displ...",16,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-10-14T15:59:16+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Jonathan Edgerley'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-19T12:22:10Z', 'versi...",NaN
2,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...","{'coding': [{'code': 'R125_Cardiology', 'displ...",22,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-10-14T15:59:16+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Jonathan Edgerley'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-19T11:51:08Z', 'versi...",NaN
3,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...",{'coding': [{'code': 'SANGER-Family-follow-up-...,61,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-12-17T12:22:52+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Christopher Campbell'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-20T10:23:30Z', 'versi...",NaN
4,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...","{'coding': [{'code': 'M80_AML_GX', 'display': ...",75,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-12-17T12:25:36+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Alistair Reid'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-21T11:24:03Z', 'versi...",NaN
5,DiagnosticReport,[{'identifier': {'assigner': {'identifier': {'...,"[{'coding': [{'code': 'GE', 'system': 'http://...","{'coding': [{'code': 'PCR-FLT3-ITD', 'display'...",88,[{'system': 'https://nw-gmsa.github.io/Id/PLAC...,2025-12-18T12:14:40+00:00,"[{'display': 'NHS North West Genomics', 'ident...","[{'contentType': 'application/pdf', 'url': 'Bi...",[{'display': 'Alisha Ram'}],final,{'identifier': {'system': 'https://fhir.nhs.uk...,"{'lastUpdated': '2025-12-21T11:37:35Z', 'versi...",{'reference': 'Encounter/86'}
